In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import serial
import time
import keyboard
from IPython.display import clear_output

In [2]:
# connect to Arduino

ser = serial.Serial('COM3', 9600, timeout=1) #check whether the Arduino is really connected with COM3 or adjust the code to the correct COM
                                             #check at settings and check whether Arduino is a connected device
time.sleep(2)

In [3]:
# read write the serial connection at a given sampling rate

def write_read(x):                  #this function takes an input x, which is a switch command and it returns data from the Arduino
    ser.write(bytes(x, 'utf-8'))    #pass the command to Arduino by the serial connection
    data = ser.readline()           #take data from Arduino
    return data                     #return the data to be processed further

In [4]:
values = []
timestamp = []

for j in range(0,2):
    num = '1'                           #power supply = True
    switchtime = time.time()

    for i in range(0,10):
        timestamp.append(time.time()- switchtime)
        clear_output(wait=True)
        value = write_read(num)
        dummy = value.decode().rstrip().split()                                     # transform the byte values that came in...
        values.append(dummy)
        #dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),dummy, 
        #dum0[0],
              '            ', end = "\r")
    
    num = '0'
    switchtime = time.time()
    
    for i in range(0,10):
        timestamp.append(time.time()- switchtime)
        clear_output(wait=True)
        value = write_read(num)
        dummy = value.decode().rstrip().split()                                     # transform the byte values that came in...
        values.append(dummy)
        # dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),value, 
        #dum0[0],
              '            ', end = "\r")
    
    
ser.close()       #close the serial port when the reading is stopped. If not done, reconnection with the Arduino is not possible    

In [5]:
df = pd.DataFrame(values, 
                  index = timestamp,
                  columns = ['S', 'V0_measured', 'V1_measured', 'V2_measured'])
df.iloc[:,1:] = df.iloc[:,1:].astype(int).div(1024/5)

In [8]:
#df.to_excel('kanweg.xlsx')

In [7]:
df.iloc[:30,:]

,S,V0_measured,V1_measured,V2_measured
0.000000,1,4.995117,4.667969,4.326172
1.025012,1,4.995117,4.746094,4.487305
2.052483,1,4.995117,4.799805,4.604492
3.078557,1,4.995117,4.848633,4.692383
4.104397,1,4.995117,4.87793,4.760742
5.127140,1,4.995117,4.907227,4.814453
6.149997,1,4.995117,4.926758,4.853516
7.169579,1,4.995117,4.936523,4.882812
8.201067,1,4.995117,4.956055,4.912109
9.226753,1,4.995117,4.960938,4.931641


In [12]:
table = None
table = pd.read_excel('kanweg.xlsx', index_col=0)
table['time'] = table.index
    
X = table.select_dtypes(include='number')
#y = pd.get_dummies(table.select_dtypes(exclude='number') )
#y.drop(columns='Health_state_H', inplace=True)

ValueError: No objects to concatenate

In [11]:
X

,S,V0_measured,V1_measured,V2_measured,time
0.000000,1,4.995117,4.667969,4.326172,0.000000
1.025012,1,4.995117,4.746094,4.487305,1.025012
2.052483,1,4.995117,4.799805,4.604492,2.052483
3.078557,1,4.995117,4.848633,4.692383,3.078557
4.104397,1,4.995117,4.877930,4.760742,4.104397
5.127140,1,4.995117,4.907227,4.814453,5.127140
6.149997,1,4.995117,4.926758,4.853516,6.149997
7.169579,1,4.995117,4.936523,4.882812,7.169579
8.201067,1,4.995117,4.956055,4.912109,8.201067
9.226753,1,4.995117,4.960938,4.931641,9.226753
